# Primeiro programa OpenGL

Este notebook apresenta os primeiros passos para criar um programa OpenGL. O nome das funções OpenGL seguem o padrão 'gl + nome_da_função'. Como o objetivo desta disciplina é focar nos conceitos de computação gráfica e não na programação com OpenGL, serão utilizadas ao longo da disciplina algumas classes que encapsulam boa parte do código OpenGL para facilitar a programação. 

## 1) Criando e configurando a janela de renderização

Durante a programação de uma aplicação gráfica, a criação da janela de renderização é a primeira coisa a ser feita. A forma como esta janela é criada muda conforme o sistema operacional (SO). Para não ter que se preocupar com estas diferenças entre os SOs, é comum utilizar bibliotecas de terceiros que permitem criar estas janelas de uma maneira que seja independente do SO. Nesta disciplina, será utilizada a biblioteca Qt para criá-las. Na função 'main' é onde a janela de renderização é criada. Essa função pode ser dividida em duas partes: a configuração do contexto OpenGL e a criação da janela em si.

O OpenGL funciona como uma máquina de estados e os dados desta máquina são armazenados no contexto OpenGL. A classe QGLFormat é utilizada para configurar este contexto com os recursos que se deseja. Neste primeiro programa, este contexto foi configurado para ser compatível com OpenGL 3.3, apenas com funcionalidade modernas (Core profile) e com 'double buffer'. O double buffer é um recurso que permite renderizar em um framebuffer enquanto um outro é utilizada para mostrar na tela uma imagem já renderizada. Quando a renderização termina, estes framebuffers trocam de lugar e o aquele que estava sendo visualizado se torna o framebuffer onde a nova renderização será realizada. Uma vez configurado o contexto OpenGL, está na hora de criar a janela de renderização com o contexto desejado.
    
O Qt fornece a classe QGLWidget que possui o básico para criar uma janela OpenGL. Para que essa classe realize a renderização desejada, ela precisa ser 'customizada' ou herdada (utilizando um jargão de orientação a objetos). A classe MyWidget é essa customização, ou seja, ela herda todas as funcionalidades de QGLWidget e adiciona o código da renderizada desejada. Basicamente, há três funções (ou métodos) que precisam ser definidas: initializeGL(), resizeGL(), paintGL(). A seguir, estes três métodos são detalhados.

## 2) Inicializando a renderização

'initializeGL()' é chamado no momento que a janela de renderização aparece na tela. É durante a execução deste método que são feitas as inicializações necessárias para renderizar a cena desejada. 

### 2.1) Definindo os dados do modelo

Neste exemplo, foi definido que o modelo a ser renderizado é constituído por 2 triângulos (6 vértices). Cada vértice possui dois atributos, posição e cor, armazenados nos vetores unidimensionais 'vertex_position' e 'vertex_color', respectivamente. As coordenadas de cada atributo são compostas por quatro componentes. Assim, tem-se uma nova coordenada a cada quatro valores tanto no vetor 'vertex_position' quanto no vetor 'vertex_color'. A coordenada de posição de cada vértice é composta pelos componentes (x, y, z, w), em que a função da componente w será explicada ao longo da disciplina. E a coordenada de cor de cada vértice é composta pelas componentes (R, G, B, A), em que a componente A representa a transparência do vértice e seu uso será explicada melhor mais adiante na disciplina.

### 2.2) Carregando os dados do modelo para dentro da GPU

Uma vez definido os dados do modelo, eles precisam ser carregados para dentro da GPU. Para realizar esta tarefa, foi criado o objeto ‘triangleRenderer’ que é da classe ModelRenderer. Essa classe recebe como parâmetros os dois vetores contendo os dados dos triângulos. Essa classe também é responsável por renderizar os objetos, como será visto mais adiante.

### 2.3) Criando o “shader program”

O shader program é o programa executado dentro da GPU. Para criar este programa é necessário dos códigos do vertex shader e do fragment shader, no mínimo. Durante a execução do programa, esses códigos são compilados e linkados a fim de criar o shader program final. A classe SimpleShaderProgram realiza esse procedimento de compilação internamente, e o vertex shader e o fragment shader dessa classe realiza nenhum processamento nos dados. Ou seja, eles simplesmente recebem os dados e os entrega para o próximo estágio do pipeline de renderização. SimpleShaderProgram possui como entrada duas variáveis de atributos de vértice, posição e cor, e o 'endereço' dessas variáveis pode ser recuperado através dos métodos 'getVertexPositionLoc()' e 'getVertexColorLoc()', respectivamente.

### 2.4) Especificando os dados do modelo como como os dados de entrada do “shader program”

Utilizando os endereços das variáveis de entrada do shader program, pode-se especificar os dados do modelo como os dados de entrada do shader program. Isso é feita através dos métodos 'setVertexPositionLoc()' e 'setVertexColorLoc()' do objeto 'triangleRenderer'.

## 3) Redimensionando a janela

O método ‘resizeGL()’ é chamado logo depois de ‘initializeGL()’ e sempre que a janela é redimensionada. ‘resizeGL()’ recebe como parâmetros a largura e a altura atuais da janela. A função ‘glViewport()’ utiliza esses valores para especificar em pixels a área da janela onde a renderização vai ser realizada. Essa função considera que a origem da janela fica no canto inferior esquerdo. Neste exemplo, ‘glViewport()’ foi utilizada para especificar que a renderização deve ocorrer na janela inteira.

## 4) Renderizando a cena

Depois da primeira vez que ‘resizeGL()’ é chamado, é a vez de ‘paintGL()’. Este método é utilizado para realizar a renderização em si e é chamado sempre que a cena renderizada precisa ser atualizada. Nesta fase, é preciso inicialmente limpar o buffer de cor do framebuffer com uma cor padrão. Esse procedimento serve para limpar valores lixo ou apagar a renderização anterior. Para isso, as funções 'glClearColor()' e 'glClear()' são utilizadas. Com a primeira função, a cor preta (0, 0, 0, 1) é configurada como a cor de background. E a segunda é a função que realmente limpa o buffer de cor (especificado como GL_COLOR_BUFFER_BIT) com a cor que foi configurada com 'glClearColor()'.

Em seguida, é iniciado o procedimento de renderização. Primeiro, é preciso especificar qual shader program será executado pela GPU. Apesar de o shader program já ter sido criado (o objeto ‘shaderProgram’), ele não está ativo na GPU. Para ativá-lo, o método 'bind()' de ‘shaderProgram’ deve ser chamado. Feito isso, os triângulos podem ser renderizados chamando o método 'render()' de 'triangleRenderer'. É durante a chamada deste método que o pipeline de renderização começa a funcionar tomando como base o shader programa ativo e utilizando os dados que o objeto 'triangleRenderer' configurou como dados de entrada. Quando a renderização termina, o shader program pode ser desativado com o método 'release()'. Se o programa tiver apenas uma shader program, a sua desativação é opcional. Por fim, é solicitado que o método 'paintGL()' seja chamado novamente através do método 'update()'.


In [5]:
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v1 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v1 import ModelRenderer

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # posição de cada vértice dos triângulos
        vertex_position = np.array([
            -0.90, -0.90, 0.0, 1.0, # Triângulo 1
             0.85, -0.90, 0.0, 1.0,
            -0.90,  0.85, 0.0, 1.0,
             0.90, -0.85, 0.0, 1.0, # Triângulo 2
             0.90,  0.90, 0.0, 1.0,
            -0.85,  0.90, 0.0, 1.0],
            dtype=np.float32)
        
        # cor de cada vértice dos triângulos
        vertex_color = np.array([
            1.0, 0.0, 0.0, 1.0, # Triângulo 1
            0.0, 1.0, 0.0, 1.0,
            0.0, 0.0, 1.0, 1.0,
            1.0, 0.0, 0.0, 1.0, # Triângulo 2
            0.0, 1.0, 0.0, 1.0,
            0.0, 0.0, 1.0, 1.0],
            dtype=np.float32)
        
        # cria um objeto responsável por carregar os dados para a GPU e renderizá-los
        self.triangleRenderer = ModelRenderer(vertex_position, vertex_color=vertex_color)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()

        # recupera os endereços das variáveis de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        color_loc = self.shaderProgram.getVertexColorLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.triangleRenderer.setVertexPositionLoc(position_loc)
        self.triangleRenderer.setVertexColorLoc(color_loc)

    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(0, 0, 0, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # inicia a renderização dos triângulos
        self.triangleRenderer.render()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()

    def resizeGL(self, width, height):
        
        # atualiza a área de renderização para ser a janela inteira
        gl.glViewport(0, 0, width, height)

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
! jupyter nbconvert --to python 17_Primeiro_programa_OpenGL.ipynb
%run -i 17_Primeiro_programa_OpenGL.py

[NbConvertApp] Converting notebook 17_Primeiro_programa_OpenGL.ipynb to python
[NbConvertApp] Writing 10453 bytes to 17_Primeiro_programa_OpenGL.py
